In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
DATA_PATH = '/dlabdata1/turkish_wiki/processed_data'

In [48]:
!ls /dlabdata1/turkish_wiki/processed_data

account_edits.csv      df_reverted_by_pageid.csv  newcomers.csv
all_registrations.csv  df_reverts.csv		  page.csv
babel.csv	       df_reverts_by_pageid.csv   page_props.csv
category.csv	       edits.csv		  redirect.csv
df_reverted.csv        flaggedpage_pending.csv


In [3]:
edits = pd.read_csv(f'{DATA_PATH}/account_edits.csv', usecols = ['event_user_id', 'event_timestamp', 'page_id'])

In [4]:
edits['event_timestamp'] = pd.to_datetime(edits['event_timestamp'],   utc = True)

In [5]:
block_dates = [pd.to_datetime('2017-04-29', utc = True), pd.to_datetime('2020-01-15 19:00', utc = True)]

In [6]:
edits.head()

,event_timestamp,event_user_id,page_id
0,2002-12-05 22:54:39+00:00,1,5
1,2003-03-13 16:30:04+00:00,6,10
2,2003-08-02 01:08:44+00:00,13,24
3,2003-08-02 01:10:08+00:00,13,24
4,2003-08-02 01:11:50+00:00,13,24


In [7]:
preblock_edits = edits[edits['event_timestamp'] < block_dates[0]]
postblock_edits = edits[edits['event_timestamp'] >= block_dates[1]]
block_edits = edits[(edits['event_timestamp'] >= block_dates[0]) & (edits['event_timestamp'] < block_dates[1])]

In [12]:
preblock_counts = preblock_edits.groupby('page_id').size()
block_counts = block_edits.groupby('page_id').size()
postblock_counts = postblock_edits.groupby('page_id').size()

In [13]:
edit_probabilities = pd.concat([preblock_counts, block_counts, postblock_counts], axis=1)

In [17]:
edit_probabilities.columns = ['preblock', 'block', 'postblock']

In [20]:
edit_probabilities = edit_probabilities.fillna(0) 

In [21]:
edit_probabilities['preblock'] = edit_probabilities['preblock']/preblock_edits.shape[0]
edit_probabilities['block'] = edit_probabilities['block']/block_edits.shape[0]
edit_probabilities['postblock'] = edit_probabilities['postblock']/postblock_edits.shape[0]

In [22]:
edit_probabilities

,preblock,block,postblock
page_id,,,
5,0.000001,0.000000,0.000000e+00
10,0.000097,0.000017,1.033032e-04
16,0.000006,0.000000,0.000000e+00
22,0.000019,0.000004,8.490676e-05
24,0.000071,0.000009,1.839647e-05
...,...,...,...
2908802,0.000000,0.000000,7.075563e-07
2908803,0.000000,0.000000,1.415113e-06
2908804,0.000000,0.000000,1.415113e-06


In [426]:
preblock_edits = edit_probabilities.preblock

In [428]:
block_edits = edit_probabilities.block

In [430]:
postblock_edits = edit_probabilities.postblock

### Get main categories for all pages

In [90]:
page = pd.read_csv(f"{DATA_PATH}/page.csv")

In [92]:
page.convert_dtypes()

,page_id,page_namespace,page_title,page_restrictions,page_is_redirect,page_is_new,page_random,page_touched,page_links_updated,page_latest,page_len,page_content_model,page_lang
0,6,2,Patrick,<NA>,0,0,0.639242802410613,20140820171459,<NA>,27775,3,wikitext,<NA>
1,10,0,Cengiz_Han,<NA>,0,0,0.00422224961614446,20210220104406,20210219225827,24884450,132792,wikitext,<NA>
2,15,1,Anasayfa,<NA>,0,0,0.00503174961081869,20210220065907,20210220065907,24907267,39869,wikitext,<NA>
3,16,0,Film_(anlam_ayrımı),<NA>,0,0,0.0762493033672211,20210115172917,20210115173623,15934440,1161,wikitext,<NA>
4,19,2,Wanli2~trwiki,<NA>,0,0,0.556826591285,20150421031350,20150421031343,15438241,6974,wikitext,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961287,2922641,3,Uğur_veysel_okşar,<NA>,0,1,0.608201327514,20210220113312,20210220113312,24908184,2792,wikitext,<NA>
1961288,2922642,0,Haltan_Aşireti,<NA>,0,0,0.951066679148,20210220113351,20210220113351,24908188,0,wikitext,<NA>
1961289,2922644,14,MÖ_1._binyılda_olaylar,<NA>,0,1,0.391064309351,20210220113335,20210220113335,24908186,21,wikitext,<NA>
1961290,2922645,14,Ümraniyespor_yabancı_futbolcuları,<NA>,0,1,0.043785518506,20210220113526,20210220113526,24908196,89,wikitext,<NA>


In [81]:
categorylinks = pd.read_csv('categorylinks.csv', delimiter = '\t', names = ['page_id', 'cat_title', 'cat_type'])

In [82]:
categorylinks['cat_title'] = categorylinks['cat_title'].str.strip("'")
categorylinks['cat_type'] = categorylinks['cat_type'].str.strip("'")

In [96]:
page = page[(page['page_namespace'] == '0') | (page['page_namespace'] == '14')][['page_id', 'page_namespace', 'page_title']]

In [314]:
page[page['page_id'] == 136717]

,page_id,page_namespace,page_title
74978,136717,0,NaN


In [85]:
page_id = 824

In [103]:
page[['page_id', 'page_namespace']] = page[['page_id', 'page_namespace']].apply(pd.to_numeric, errors='coerce').convert_dtypes()

In [107]:
pages = dict(zip(page[page['page_namespace'] == 0].page_id, page[page['page_namespace'] == 0].page_title))

In [108]:
categories = dict(zip(page[page['page_namespace'] == 14].page_id, page[page['page_namespace'] == 14].page_title))

In [113]:
categorylinks[['page_id']] = categorylinks[['page_id']].apply(pd.to_numeric, errors='coerce').convert_dtypes()

In [162]:
categorylinks.cat_title = categorylinks.cat_title.str.replace("\\", "")

In [304]:
merged_cat_links = pd.merge(categorylinks, page,  left_on = 'cat_title', right_on = 'page_title')

In [305]:
merged_cat_links = pd.merge(merged_cat_links, page, left_on = 'page_id_x', right_on = 'page_id')

In [306]:
merged_cat_links

,page_id_x,cat_title,cat_type,page_id_y,page_namespace_x,page_title_x,page_id,page_namespace_y,page_title_y
0,10,1162_doğumlular,page,62412,14,1162_doğumlular,10,0,Cengiz_Han
1,10,1227_yılında_ölenler,page,62540,14,1227_yılında_ölenler,10,0,Cengiz_Han
2,10,Attan_düşerek_ölenler,page,2408001,14,Attan_düşerek_ölenler,10,0,Cengiz_Han
3,10,BIBSYS_tanımlayıcısı_olan_Vikipedi_maddeleri,page,1932816,14,BIBSYS_tanımlayıcısı_olan_Vikipedi_maddeleri,10,0,Cengiz_Han
4,10,BNC_tanımlayıcısı_olan_Vikipedi_maddeleri,page,2615013,14,BNC_tanımlayıcısı_olan_Vikipedi_maddeleri,10,0,Cengiz_Han
...,...,...,...,...,...,...,...,...,...
3856980,2921445,Vikiproje_II._Dünya_Savaşı_Taslak-sınıf_sayfalar,subcat,2921431,14,Vikiproje_II._Dünya_Savaşı_Taslak-sınıf_sayfalar,2921445,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_T...
3856981,2921452,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_s...,subcat,2921844,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_s...,2921452,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_C...
3856982,2921452,Vikiproje_II._Dünya_Savaşı_C-sınıf_sayfalar,subcat,2921429,14,Vikiproje_II._Dünya_Savaşı_C-sınıf_sayfalar,2921452,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_C...
3856983,2921839,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_s...,subcat,2921844,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_s...,2921839,14,Vikiproje_II._Dünya_Savaşı_bilinmeyen-önemli_d...


In [307]:
merged_cat_links = merged_cat_links[['page_id_x', 'page_title_y',
       'page_id_y', 'page_title_x']]

In [308]:
merged_cat_links.columns = ['child_id', 'child_name', 'parent_id', 'parent_name']

In [309]:
## Remove self loops
merged_cat_links = merged_cat_links[~(merged_cat_links['child_id'] == merged_cat_links['parent_name'])]

In [302]:
category_links = list(zip(merged_cat_links.parent_id, merged_cat_links.child_id))

In [199]:
import pickle

In [200]:
pickle.dump(category_links, open("links.p", "wb" ))

In [194]:
roots = set()
mapping = {}
for parent,child in category_links:
    childitem = mapping.get(child,None)
    if childitem is None:
        childitem =  {}
        mapping[child] = childitem
    else:
        roots.discard(child)
    parentitem = mapping.get(parent,None)
    if parentitem is None:
        mapping[parent] = {child:childitem}
        roots.add(parent)
    else:
        parentitem[child] = childitem

tree = {id : mapping[id] for id in roots}

print (tree)

KeyboardInterrupt: 

In [312]:
merged_cat_links = merged_cat_links[merged_cat_links['child_name'] != 'İçindekiler']

In [317]:
merged_cat_links = merged_cat_links[~merged_cat_links.child_name.isna()]

In [328]:
merged_cat_links[merged_cat_links['parent_id'] == 742060]

,child_id,child_name,parent_id,parent_name
246,68454,William_Faulkner,742060,Nobel_Edebiyat_Ödülü_sahipleri
4394,6200,Samuel_Beckett,742060,Nobel_Edebiyat_Ödülü_sahipleri
8061,12448,Albert_Camus,742060,Nobel_Edebiyat_Ödülü_sahipleri
8628,13219,Ernest_Hemingway,742060,Nobel_Edebiyat_Ödülü_sahipleri
12103,18204,Knut_Hamsun,742060,Nobel_Edebiyat_Ödülü_sahipleri
...,...,...,...,...
2922822,678167,Jean-Marie_Gustave_Le_Clézio,742060,Nobel_Edebiyat_Ödülü_sahipleri
3151387,285834,Gerhart_Hauptmann,742060,Nobel_Edebiyat_Ödülü_sahipleri
3188318,39042,Dario_Fo,742060,Nobel_Edebiyat_Ödülü_sahipleri
3252529,126332,Selma_Lagerlöf,742060,Nobel_Edebiyat_Ödülü_sahipleri


In [214]:
# Ana madde konulari = Main_topic_articles
page[page['page_title'] == 'Ana_madde_konuları']

,page_id,page_namespace,page_title
1640645,2570808,14,Ana_madde_konuları


In [212]:
mapping = pickle.load(open( "mapping2.p", "rb" ) )

In [215]:
main_topic_ids = mapping[2570808].keys()

In [222]:
for key in main_topic_ids:
    print(f"{page[page['page_id'] == key].page_title.values[0]}: ")
    subcat_keys = mapping[key].keys()
    print(f"     {page[page['page_id'].isin(subcat_keys)].page_title.values}")

Dil: 
     ['Diller' 'Dilbilim' 'İyelik_eki' 'Anadili' 'Dil_sınavları'
 'Dil_ile_ilgili_ödüller' 'Dil_eğitimi' 'Dil_şablonları' 'Sözlü_iletişim'
 'Konuşma_dili' 'Doğal_dil_ve_bilgisayar' 'Kendine_göndergeli_önerme'
 'Dil_yazılımları' 'Konuşma_ve_dil_patolojisi' 'Noktalama'
 'Dil_birlikleri' 'Önerme' 'Doğal_dil' 'Cinsiyet_ve_dil' 'Aelius_Donatus'
 'Küçük_dil' 'Ergatif_dil' 'Kelime_bilgisi']
Spor: 
     ['Sporcular' 'Spor_adamları' 'Spor_taslakları' 'Sporlar' 'Spor_terimleri'
 'Spor_şablonları' 'Spor_listeleri' 'Spor_tarihi' 'Spor_ödülleri'
 'Spor_mekânları' 'Gençler_spor' 'Spor_kültürü' 'Kurguda_spor'
 'Spor_kuralları_ve_yönetmelikleri' 'Spor_istatistikleri_ve_rekorları'
 'Spor_etkinlikleri' 'Spor_portalleri' 'Spor_ve_siyaset'
 'Spor_ekipmanları' 'Spor_bilimi' 'Spor_kuruluşları' 'Spor_tartışmaları'
 'Spor_meslekleri' 'Cinsiyet_ve_spor' 'Gösteri_sporu'
 'Spor_giyim_markaları' 'Yerlerine_göre_spor' 'Fitness_merkezi'
 'Döneme_göre_spor' 'Spor_giyim' 'Spor_modası'
 'GGO_Futbol_bölümleri_lis

In [349]:
def dfs(visited, graph, node):
    if node not in visited:
        visited.add(node)
        for neighbour in graph[node]:
            dfs(visited, graph[node], neighbour)

In [361]:
main_topic_dict = {}
for key in main_topic_ids:
    visited = set()
    dfs(visited, mapping[2570808], key)
    main_topic_dict[key] = visited

In [390]:
topic_df = page.copy()
for key, values in main_topic_dict.items():
    topic_name = topic_df[topic_df.page_id == key].page_title.values[0]
    topic_df.loc[topic_df.page_id.isin(list(values)), topic_name] = True

In [396]:
topic_df = topic_df.fillna(value = False)

In [225]:
main_categories = mapping[12539].keys()

In [226]:
for key in main_categories:
    print(f"{page[page['page_id'] == key].page_title.values[0]}: ")
    subcat_keys = mapping[key].keys()
    print(f"     {page[page['page_id'].isin(subcat_keys)].page_title.values}")

Bilim: 
     ['Bilim' 'Teknoloji' 'Sözdebilim' 'Bilimsel_yöntem' 'Bilim_tarihi'
 'Bilim_taslakları' 'Bilim_felsefesi' 'Bilim_listeleri' 'Bilimsel_gözlem'
 'Bilimsel_terminoloji' 'Bilim_dalları' 'Bilim_şablonları'
 'Bilim_ve_mühendislik_ödülleri' 'Bilim_ve_toplum' 'Bilimsel_teoriler'
 'Bilimsel_yasalar' 'Kıtalara_göre_bilim_ve_teknoloji' 'Bilim_portalleri'
 'Ülkelere_göre_bilim_ve_teknoloji' 'Bilim_meslekleri' 'Bilim_yapıları'
 'Bilimsel_spekülasyonlar' 'Bilimsel_adlandırma' 'Bilimsel_olaylar'
 'Bilim_olayları' 'Bilim_insanları' 'Döneme_göre_bilim' 'Jeomühendislik'
 'Oktay_Güvemli' 'Matematik_tarihi' 'Felsefe-Bilim' 'Silahlanma']
Kavramlar: 
     ["Ockham'ın_usturası" 'Bilgi' 'Doğa' 'Bilişsel_şema' 'Organik' 'Terimler'
 'Teoriler' 'Bilinç_dışı' 'Öznellik' 'Sözcükler' 'Entelektüel' 'Kategori'
 'Sistemler' 'Popüler' 'Kendi_kendini_üretim' 'Prototip' 'Yapı'
 'Kavram_haritası' 'Kavram' 'Oryantasyon' 'Kavramsal_sistemler'
 'Kavramsal_modeller' 'Sahalara_göre_kavramlar' 'Kalite'
 'Mr._Magoo_E

In [362]:
main_categories_dict = {}
for key in main_categories:
    visited = set()
    dfs(visited, mapping[12539], key)
    main_categories_dict[key] = visited

In [397]:
category_df = page.copy()
for key, values in main_categories_dict.items():
    category_name = category_df[category_df.page_id == key].page_title.values[0]
    category_df.loc[category_df.page_id.isin(list(values)), category_name] = True

In [398]:
category_df = category_df.fillna(value = False)

In [401]:
category_df[category_df.page_namespace == 0]

,page_id,page_namespace,page_title,Bilim,Kavramlar,İnanç,Kültür,Hukuk,Siyaset,Doğa,...,Sağlık,İşletme,Kronoloji,Çevre,Nesneler,Olaylar,Dil,Tarım,Bilim_ve_teknoloji,Ana_madde_konuları
1,10,0,Cengiz_Han,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,16,0,Film_(anlam_ayrımı),True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,22,0,Mustafa_Suphi,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
6,24,0,Linux,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,25,0,MHP,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961272,2922626,0,Sosyal_faşizm,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1961276,2922630,0,Susie_Dietter,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1961279,2922633,0,Ev_zencisi,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1961288,2922642,0,Haltan_Aşireti,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [405]:
category_df[['Bilim', 'Kavramlar',
       'İnanç', 'Kültür', 'Hukuk', 'Siyaset', 'Doğa', 'Kişiler', 'Sağlık',
       'İşletme', 'Kronoloji', 'Çevre', 'Nesneler', 'Olaylar', 'Dil', 'Tarım',
       'Bilim_ve_teknoloji', 'Ana_madde_konuları']].sum(axis = 1).describe()

count    919081.000000
mean         12.668396
std           8.218451
min           0.000000
25%           0.000000
50%          18.000000
75%          18.000000
max          18.000000
dtype: float64

In [408]:
topic_df[['Dil', 'Spor', 'Ekonomi',
       'Matematik', 'Din', 'Hukuk', 'Coğrafya', 'Bilim', 'Sağlık', 'Teknoloji',
       'Eğitim', 'Enerji', 'İşletme', 'Kültür', 'Bilgi', 'Zihin', 'Doğa',
       'Evren', 'Müzik', 'Siyaset', 'Tarih', 'Hükûmet', 'Yaşam', 'Suç',
       'İçecek', 'Beşeri_bilimler', 'Kuruluş', 'Eğlence', 'Kavram', 'Yiyecek',
       'Felsefe', 'Fiziksel_cisim', 'Kişi', 'Akademik_disiplin', 'Küçük_dil',
       'Haksız_tahrik']].sum(axis = 1).describe()

count    919081.000000
mean         19.002604
std          12.327684
min           0.000000
25%           0.000000
50%          27.000000
75%          27.000000
max          28.000000
dtype: float64

In [416]:
topic_df[topic_df['page_id'] == 40]

,page_id,page_namespace,page_title,Dil,Spor,Ekonomi,Matematik,Din,Hukuk,Coğrafya,...,Kuruluş,Eğlence,Kavram,Yiyecek,Felsefe,Fiziksel_cisim,Kişi,Akademik_disiplin,Küçük_dil,Haksız_tahrik
14,40,0,Beşiktaş_JK,True,True,True,True,True,True,True,...,False,True,False,False,True,False,False,False,False,False


In [417]:
category_df[category_df['page_id'] == 40]

,page_id,page_namespace,page_title,Bilim,Kavramlar,İnanç,Kültür,Hukuk,Siyaset,Doğa,...,Sağlık,İşletme,Kronoloji,Çevre,Nesneler,Olaylar,Dil,Tarım,Bilim_ve_teknoloji,Ana_madde_konuları
14,40,0,Beşiktaş_JK,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [423]:
mapping[2570808][79].keys()

dict_keys([12219, 41697, 46164, 78372, 113121, 116139, 147338, 378934, 630996, 631958, 918834, 974840, 989892, 1000345, 1176151, 1176214, 1203982, 1243711, 1253633, 1442503, 1470105, 1607389, 1705770, 1757570, 2027742, 2029346, 2029603, 2380465, 2797057, 2799933, 2799937, 2842538, 2848039, 2884496, 2894839, 2912786, 2920047, 2922362])

In [425]:
page[page['page_id'].isin([12219, 41697, 46164, 78372, 113121, 116139, 147338, 378934, 630996, 631958, 918834, 974840, 989892, 1000345, 1176151, 1176214, 1203982, 1243711, 1253633, 1442503, 1470105, 1607389, 1705770, 1757570, 2027742, 2029346, 2029603, 2380465, 2797057, 2799933, 2799937, 2842538, 2848039, 2884496, 2894839, 2912786, 2920047, 2922362])]

,page_id,page_namespace,page_title
6224,12219,14,Sporcular
23007,41697,14,Spor_adamları
25049,46164,14,Spor_taslakları
42721,78372,14,Sporlar
62045,113121,14,Spor_terimleri
63776,116139,14,Spor_şablonları
80642,147338,14,Spor_listeleri
215349,378934,14,Spor_tarihi
329935,630996,14,Spor_ödülleri
330427,631958,14,Spor_mekânları
